In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
#!/bin/bash
! kaggle datasets download ckay16/accident-detection-from-cctv-footage

Dataset URL: https://www.kaggle.com/datasets/ckay16/accident-detection-from-cctv-footage
License(s): ODbL-1.0


In [3]:
!unzip -q '/content/accident-detection-from-cctv-footage.zip'

In [4]:
import os
import shutil

# Paths
base_path = "/content/data"
sets = ["train", "val"]
categories = ["Accident", "Non Accident"]

# Create target directories
for s in sets:
    os.makedirs(f"{base_path}/{s}/images", exist_ok=True)
    os.makedirs(f"{base_path}/{s}/labels", exist_ok=True)

In [9]:
!pip install ultralytics opencv-python-headless --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.4 MB/s eta 0:00:00


In [10]:
# Import Libraries
import os
import cv2
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Define Helper Function to Create Labels (full image as bbox)
def create_labels(image_folder, label_folder, class_id):
    os.makedirs(label_folder, exist_ok=True)

    for img_name in os.listdir(image_folder):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(image_folder, img_name)
            label_path = os.path.join(label_folder, img_name.rsplit('.', 1)[0] + '.txt')

            # Read the image to get dimensions
            img = cv2.imread(img_path)
            height, width, _ = img.shape

            # Full image = bbox covering 100%
            x_center = 0.5
            y_center = 0.5
            bbox_width = 1.0
            bbox_height = 1.0

            with open(label_path, 'w') as f:
                f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

#  Generate Labels for All Images

train_accident = '/content/data/train/Accident'
train_non_accident = '/content/data/train/Non Accident'
val_accident = '/content/data/val/Accident'
val_non_accident = '/content/data/val/Non Accident'

# Create labels for Train
create_labels(train_accident, '/content/data/train/labels', class_id=1)
create_labels(train_non_accident, '/content/data/train/labels', class_id=0)

# Create labels for Validation
create_labels(val_accident, '/content/data/val/labels', class_id=1)
create_labels(val_non_accident, '/content/data/val/labels', class_id=0)

print("Labels created successfully!")

✅ Labels created successfully!


In [7]:
# Move files and delete original folders
for s in sets:
    for cat in categories:
        src_dir = os.path.join(base_path, s, cat)
        dst_dir = os.path.join(base_path, s, "images")

        if os.path.exists(src_dir):
            for filename in os.listdir(src_dir):
                src_file = os.path.join(src_dir, filename)
                dst_file = os.path.join(dst_dir, filename)
                if os.path.isfile(src_file):
                    shutil.move(src_file, dst_file)
            # Remove the empty category folder
            shutil.rmtree(src_dir)

print("Files moved and folders cleaned up.")

Files moved and folders cleaned up.


In [ ]:
# !cp -r /content/data /content/drive/MyDrive/acci_data

In [8]:
# Create Dataset YAML file
dataset_yaml = """
train: /content/data/train
val: /content/data/val

nc: 2
names: ['Non-Accident', 'Accident']
"""

with open('/content/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

print("dataset.yaml created successfully!")

✅ dataset.yaml created successfully!


In [12]:
# Train YOLOv8 Model
model = YOLO('yolov8n.pt')  # 'n' is nano, you can also try 's' for small version

# Start training
model.train(
    data='/content/dataset.yaml',
    epochs=50,
    imgsz=416,
    batch=16,
    patience=7,
    optimizer='Adam',
    device=0,
    name='accident_yolov8_model',
    workers=2
)

print("Training Completed!")

Ultralytics 8.3.125 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset.yaml, epochs=50, time=None, patience=7, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=accident_yolov8_model2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

train: Scanning /content/data/train/labels.cache... 791 images, 0 backgrounds, 0 corrupt: 100%|██████████| 791/791 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1125.0±887.9 MB/s, size: 256.7 KB)


val: Scanning /content/data/val/labels.cache... 98 images, 0 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]


...

      16/50      1.15G     0.2112     0.8261     0.8755         22        416: 100%|██████████| 50/50 [00:10<00:00,  4.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.54it/s]

                   all         98         98      0.495          1      0.551      0.551
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



16 epochs completed in 0.052 hours.
Optimizer stripped from runs/detect/accident_yolov8_model2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/accident_yolov8_model2/weights/best.pt, 6.2MB

Validating runs/detect/accident_yolov8_model2/weights/best.pt...
Ultralytics 8.3.125 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.38it/s]


                   all         98         98      0.468          1      0.618      0.613
          Non-Accident         52         52      0.468          1      0.592      0.586
              Accident         46         46      0.469          1      0.644      0.641
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/accident_yolov8_model2
Training Completed!


In [ ]:
# shutil.make_archive("/content/data_zip", 'zip', "/content/runs/detect/accident_yolov8_model2")

'/content/data_zip.zip'